In [1]:
import torch
from torch import nn as nn
from models.nac import NAC
from models.nalu import NALU
from utilities.train_utils import get_eval_loss, get_eval_preds
from utilities.train_utils import train
from torch.optim import RMSprop, Adam
import numpy as np
import pandas as pd

### Function to create toy dataset

In [2]:
def make_data(min_val, max_val, num_obs, op):
    '''
    Generates toy data and target by sampling values from a
    uniform distribution parameterized by min_val and max_val.
    '''
    data = np.random.uniform(min_val, max_val, size=(num_obs, 2))
    if op == '+':
        targets = data[:, 0] + data[:, 1]
    elif op == '-':
        targets = data[:, 0] - data[:, 1]
    elif op == '*':
        targets = data[:, 0] * data[:, 1]
    elif op == '/':
        targets = data[:, 0] / data[:, 1]
    elif op == '^2':
        data = np.random.uniform(min_val, max_val, size=(num_obs, 1))
        targets = data ** 2
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size=(num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

### Function to compute accuracy of predictions

In [3]:
def accuracy_score(preds, targets, tol=1e-3):
    '''
    Computes prediction accuracy by checking if
    predictions are equal to the target upto `tol`
    places after decimal.
    '''
    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    accuracy = np.isclose(preds, targets, rtol=tol)
    accuracy = accuracy.astype(np.int32).mean()
    return accuracy * 100

### Running experiments

In [5]:
eps = 1e-12
ops = ['+', '-', '*', '/', 'sqrt', '^2']
test_scores = {}

for op in ops:
    X_train, Y_train = make_data(10, 20, 10000, op)
    X_valid, Y_valid = make_data(10, 20, 2000, op)
    X_test, Y_test = make_data(0, 30, 10000, op) # Test set contains both interpolated
                                                 # and extrapolated data
    
    if op == '^2':
        model = nn.Sequential(NALU(X_train.shape[1], 2), NALU(2, 1))
    else:
        model = NALU(X_train.shape[1], 1)
    model = train(model, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, patience=15, batch_size=32)
    
    test_preds, test_targets = get_eval_preds(model, X_test,
                                              Y_test, 32, False)
    
    test_acc = accuracy_score(test_preds, test_targets)
    
    print('Test accuracy for [{}]: {:.2f}%'.format(op, test_acc))
    
    if op == 'sqrt':
        test_scores['sqrt(a)'] = test_acc
    elif op == '^2':
        test_scores['a ^ 2'] = test_acc
    else:
        test_scores['a '+op+' b'] = test_acc

[Epoch: 1] Training loss after 47 batches: 1.737:  15%|█▍        | 46/313 [00:00<00:01, 253.07it/s]     

Test accuracy for [+]: 100.00%


[Epoch: 1] Training loss after 56 batches: 189.030:  18%|█▊        | 55/313 [00:00<00:00, 280.86it/s]    

Test accuracy for [-]: 99.98%


[Epoch: 1] Training loss after 47 batches: 0.292:  15%|█▍        | 46/313 [00:00<00:01, 251.30it/s]    

...

[Epoch: 1] Training loss after 55 batches: 0.477:  17%|█▋        | 54/313 [00:00<00:00, 282.55it/s]        

Test accuracy for [/]: 98.90%


[Epoch: 1] Training loss after 49 batches: 231.126:  15%|█▌        | 48/313 [00:00<00:01, 250.78it/s]   

Test accuracy for [sqrt]: 100.00%


Test accuracy for [^2]: 100.00%


### Saving the results

In [6]:
results = pd.DataFrame.from_dict(test_scores, orient='index', columns=['Accuracy'])
results.to_csv('results.csv', index=False)
results.head(6)

Accuracy
a + b      100.00
a - b       99.98
a * b      100.00
a / b       98.90
sqrt(a)    100.00
a ^ 2      100.00